In [ ]:
import pandas as pd
import numpy as np
from glob import glob

In [ ]:
comuni_to_drop = [22111,22152,22046,22030,22027,22154,22070,22080,22063,22088,22041,22211,22253,22254,22252,41032,97085]
provincia_to_capoluogo = {
	"Valle d'Aosta/Vallée d'Aoste" : "Aosta",
	"Forlì-Cesena" : "Forlì",
	"Pesaro e Urbino" : "Pesaro",
	"Massa-Carrara" : "Massa",
	"Verbano-Cusio-Ossola" : "Verbania",
	"Monza e della Brianza" : "Monza",
	"Barletta-Andria-Trani" : "Andria",
	"Sud Sardegna" : "Carbonia"
}

In [ ]:
# loading only the relevant entries
comuni = pd.read_csv("../raw/Mobility/Italy/POSAS_2020_it_Comuni.csv", skiprows=1, delimiter=";", usecols=["Codice comune", "Comune", "Età", "Totale"])
# selecting population with any age
comuni = comuni.loc[comuni["Età"] == 999]
comuni.drop(columns="Età", inplace=True)
# dropping the municipalities that have appeared/disappeared
comuni = comuni.loc[~comuni["Codice comune"].isin(comuni_to_drop)]
# computing the province code
comuni["Codice provincia"] = comuni["Codice comune"] // 1000
# sorting
comuni.sort_values("Codice comune", inplace=True)
# visualizing
comuni

In [ ]:
province = pd.read_csv("../raw/Mobility/Italy/Popolazione residente.csv", usecols=["Codice provincia", "Provincia"])
province.set_index("Codice provincia", inplace=True)
province["Capoluogo"] = province["Provincia"].replace(provincia_to_capoluogo)
province["Codice capoluogo"] = comuni.set_index("Comune").loc[province["Capoluogo"]].set_index("Codice provincia")["Codice comune"]
province["Popolazione"] = comuni[["Totale","Codice provincia"]].groupby("Codice provincia").sum()
province

In [ ]:
province_IDs = province.reset_index()
province_IDs.index.name = "Provincia ID"
province_IDs[["Codice provincia","Capoluogo"]].to_csv("patchIDs/italy.csv", index=True)

In [ ]:
thetypes = {"OR_PROCOM":"u4","OR_PROV":"u4","DEST_PROCOM":"u4","DEST_PROV":"u4","KM_TOT":"f8"}
distances = []
for filename in sorted(glob("../raw/Mobility/Italy/Distanze/*.csv")):
	spacedata = pd.read_csv(filename, delimiter=";", decimal=",", usecols=thetypes.keys(), dtype=thetypes)
	spacedata = spacedata.loc[np.logical_and(spacedata["OR_PROCOM"].isin(province["Codice capoluogo"]), spacedata["DEST_PROCOM"].isin(province["Codice capoluogo"]))]
	spacedata.drop(columns=["OR_PROCOM","DEST_PROCOM"], inplace=True)
	distances.append(spacedata)
distances = pd.concat(distances).pivot(columns="DEST_PROV", index="OR_PROV")
distances = distances.droplevel(0, axis="columns")
distances

In [ ]:
province["Popolazione"].to_csv("Ns/italy.csv", index=False)
distances.to_csv("distances/italy.csv", index=False)

In [ ]:
(province["Popolazione"]/100).round().astype(int).to_csv("Ns/italy_mini.csv", index=False)
(province["Popolazione"]/1000).round().astype(int).to_csv("Ns/italy_micro.csv", index=False)